## 1. Installing libraries

In [ ]:
!pip install "tensorflow<2.11" opencv_python matplotlib

## 2. Importing Dependencies

In [ ]:
import cv2
import os
import numpy as np
import random
import matplotlib.pyplot as plt

#tensoeflow dependencies
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, MaxPooling2D, Flatten, Dense, Input
import tensorflow as tf

#for unique image name
import uuid

## 3. Setting GPU Growth

In [ ]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

## 4. File Structure

In [ ]:
#making paths for poitives, negatives and anchors
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [ ]:
#making directories for positves, negatives and anchors
os.makedirs(POS_PATH, exist_ok=True)
os.makedirs(NEG_PATH, exist_ok=True)
os.makedirs(ANC_PATH, exist_ok=True)

## 5. Importing the Dataset

### 5.1 Negatives

In [ ]:
#uncompressing the lfw dataset
!tar xf lfw.tgz

#move the lfw images to the negatives
for directory in os.listdir('lfw'):
  for file in os.listdir(os.path.join('lfw', directory)):
    EX_Path = os.path.join('lfw', directory, file)
    NEW_Path = os.path.join(NEG_PATH, file)
    os.replace(EX_Path, NEW_Path)

### 5.2 Positives and Anchors

In [ ]:
# Load the Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the camera
cap = cv2.VideoCapture(1)  # Use 0 to access the default webcam

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break

    #Convert to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    for (x, y, w, h) in faces:
        #Crop the face from the frame
        face = frame[y:y+h, x:x+w]

        #Resize the face to 250x250 pixels
        face_resized = cv2.resize(face, (250, 250))

        #Show the resized face
        cv2.imshow("Image", Image)

        #Save as anchor image
        if cv2.waitKey(1) & 0xFF == ord("a"):
            image = os.path.join(ANC_PATH, "{}.jpg".format(uuid.uuid1()))
            cv2.imwrite(image, face_resized)

        #Save as positive image
        if cv2.waitKey(1) & 0xFF == ord("p"):
            image = os.path.join(POS_PATH, "{}.jpg".format(uuid.uuid1()))
            cv2.imwrite(image, face_resized)

    #Exit when 'e' key is pressed
    if cv2.waitKey(1) & 0xFF == ord("e"):
        break

cap.release()
cv2.destroyAllWindows()

### 5.3 Data Augmentation (used in siamese_modelv2.h5)

In [ ]:
def augment(img):
    data = []
    for i in range(5):
        img = tf.image.stateless_random_brightness(img, max_delta = 0.02, seed=(1,2))
        img = tf.image.stateless_random_flip_left_right(img, seed = (np.random.randint(100), np.random.randint(100)))
        img = tf.image.stateless_random_contrast(img, lower = 0.6, upper = 1, seed = (1,3))
        img = tf.image.stateless_random_jpeg_quality(img, min_jpeg_quality=90, max_jpeg_quality=100, seed = (np.random.randint(100), np.random.randint(100)))
        img = tf.image.stateless_random_saturation(img, lower = 0.9, upper = 1, seed = (np.random.randint(100), np.random.randint(100)))
        data.append(img)
    return data

In [ ]:
#some less brighter images
for file in os.listdir(os.path.join(ANC_PATH)):
    img = cv2.imread(os.path.join(ANC_PATH, file))
    img_lb = tf.image.adjust_brightness(img, -0.2)
    cv2.imwrite(os.path.join(ANC_PATH, "{}.jpg".format(uuid.uuid1())), img_lb.numpy())
#adding augmented images to anchor folder
for file in os.listdir(os.path.join(ANC_PATH)):
    img = cv2.imread(os.path.join(ANC_PATH, file))
    augmented_images = augment(img)
    for img in augmented_images:
        cv2.imwrite(os.path.join(ANC_PATH, "{}.jpg".format(uuid.uuid1())), img.numpy())

In [ ]:
#some less brighter images
for file in os.listdir(os.path.join(POS_PATH)):
    img = cv2.imread(os.path.join(POS_PATH, file))
    img_lb = tf.image.adjust_brightness(img, -0.2)
    cv2.imwrite(os.path.join(POS_PATH, "{}.jpg".format(uuid.uuid1())), img_lb.numpy())
#adding augmented images to anchor folder
for file in os.listdir(os.path.join(POS_PATH)):
    img = cv2.imread(os.path.join(POS_PATH, file))
    augmented_images = augment(img)
    for img in augmented_images:
        cv2.imwrite(os.path.join(POS_PATH, "{}.jpg".format(uuid.uuid1())), img.numpy())

## 6. Loading and Preprocessing the Dataset(Images)

### 6.1 Loading the Images

In [ ]:
anchors = tf.data.Dataset.list_files(ANC_PATH+"\*.jpg").take(2500)
positives = tf.data.Dataset.list_files(POS_PATH+"\*.jpg").take(2500)
negatives = tf.data.Dataset.list_files(NEG_PATH+"\*.jpg").take(2500)

### 6.2 Preprocessing the Images

In [ ]:
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (100, 100))
    img = img/255
    return img

### 6.3 Creating Labeled Dataset

In [ ]:
positives = tf.data.Dataset.zip((anchors, positives, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchors)))))
negatives = tf.data.Dataset.zip((anchors, negatives, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchors)))))
dataset = positives.concatenate(negatives)

### 6.4 Train and Test Partition

In [ ]:
#preprocessing the labelled poitives and negatives
def preprocess_two(input_image, validation_image, label):
    return preprocess(input_image), preprocess(validation_image), label

In [ ]:
#dataset
dataset = dataset.map(preprocess_two)
dataset = dataset.cache()
dataset = dataset.shuffle(buffer_size = 10000)

In [ ]:
#training set
train_data = dataset.take(round(len(dataset)*0.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [ ]:
#test_set
test_data = dataset.skip(round(len(dataset)*0.7))
test_data = test_data.take(round(len(dataset)*0.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

## 7. Creating the Model

### 7.1 Creating Embedding layer

In [ ]:
def embedding():
    #1st block
    inp = Input(shape = (100, 100, 3), name = "Input Layer")
    c1 = Conv2D(64, (10, 10), activation = "relu")(inp)
    m1 = MaxPooling2D(64, (2,2), padding = "same")(c1)
                      
    #2nd block
    c2 = Conv2D(128, (7,7), activation = "relu")(m1)
    m2 = MaxPooling2D(64, (2,2), padding = "same")(c2)
    
    #3rd block
    c3 = Conv2D(128, (4,4), activation = "relu")(m2)
    m3 = MaxPooling2D(64, (2,2), padding = "same")(c3)
    
    #4th block
    c4 = Conv2D(256, (4,4), activation = "relu")(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation = "sigmoid")(f1)

    return Model(inputs = [inp], outputs = [d1], name = "embedding")                  

In [ ]:
embedding_model = embedding()

In [ ]:
embedding_model.summary()

### 7.2 Creating Siamese Distance Layer

In [ ]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

### 7.3 Combining the above two to make the Siamese Model

In [ ]:
def make_siamese_model():
    input_image = Input(shape = (100, 100, 3), name = "input_img")
    validation_image = Input(shape = (100, 100, 3), name = "validation_img")

    siamese_layer = L1Dist()
    distances = siamese_layer(embedding_model(input_image), embedding_model(validation_image))

    classifier = Dense(1, activation = "sigmoid")(distances)
    return Model(inputs = [input_image, validation_image], outputs = [classifier], name = "siamese_model")

In [ ]:
siamese_model = make_siamese_model()

In [ ]:
siamese_model.summary()

## 8. Training the Model

### 8.1 Setting up the Loss and Optimizer Functions

In [ ]:
binary_cross_entropy = tf.keras.losses.BinaryCrossentropy()

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4)

### 8.2 Making the Checkpoints directory

In [ ]:
checkpoints_path = "./training_checkpoints"
checkpoints_prefix = os.path.join(checkpoints_path, "ckpt")
checkpoints = tf.train.Checkpoint(optimizer = optimizer, siamese_model = siamese_model)

### 8.3 Building a Function to find and apply Gradients

In [ ]:
@tf.function
def gradients(batch):
    with tf.GradientTape() as tape:
        X = batch[:2]
        y = batch[2]
        yhat = siamese_model(X, training = True)
        loss = binary_cross_entropy(yhat, y)
    grads = tape.gradient(loss, siamese_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, siamese_model.trainable_variables))
    return loss

### 8.4 Creating the Training loop Function

In [ ]:
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
def train(data, EPOCHS):
    for epoch in range(1, EPOCHS+1):
        print("\n Epoch {}/{}".format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))

        r = Recall()
        p = Precision()

        for idx, batch in enumerate(data):
            # Run train step here
            loss = gradients(batch)
            yhat = siamese_model.predict(batch[:2], verbose = 0)
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat) 
            progbar.update(idx+1)
        print(loss.numpy(), r.result().numpy(), p.result().numpy())
        if epoch%10 == 0:
            checkpoints.save(file_prefix = checkpoints_prefix)
            

In [ ]:
train(train_data, 5)

## 9. Evaluating the Model

### 9.1 Importing the Metrics

In [ ]:
from tensorflow.keras.metrics import Precision, Recall

### 9.2 Calculating Metrics

In [ ]:
r = Recall()
p = Precision()
for test_input, test_validation, ytrue in test_data.as_numpy_iterator():
    yhat = siamese_model.predict([test_input, test_validation], verbose = 0)
    r.update_state(ytrue, yhat)
    p.update_state(ytrue, yhat)
print(r.result().numpy(), p.result().numpy())

## 10. Saving the Model

In [ ]:
siamese_model.save("models/siamese_modelv2.h5")

In [ ]:
model = tf.keras.models.load_model("models/siamese_modelv2.h5", custom_objects = {"L1Dist":L1Dist, "BinaryCrossentrpy":tf.keras.losses.BinaryCrossentropy})

## 11. Real Time Verification

### 11.1 Verification Function

In [ ]:
def verification(model, detection_threshold, verification_threshold):
    results = []
    for image in os.listdir(os.path.join("application_data", "verification_images")):
        input_image = preprocess(os.path.join("application_data", "input_image", "input_image.jpg"))
        verification_image = preprocess(os.path.join("application_data", "verification_images", image))

        #prediction for the results
        result = model.predict(list(np.expand_dims([input_image, verification_image], axis = 1)), verbose = 0)
        results.append(result)

    #verifying
    detection = np.sum(np.array(results)>detection_threshold)
    verification = detection/len(os.listdir(os.path.join("application_data", "verification_images")))
    verified = verification>verification_threshold

    return results, verified

### 11.2 Webcam Capture and Verification

In [ ]:
# Load the Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(1)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break

    #Convert to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        #Crop the face from the frame
        face = frame[y:y+h, x:x+w]

        #Resize the face to 250x250 pixels
        face_resized = cv2.resize(face, (250, 250))

        #Show the resized face
        cv2.imshow("Verification", Image)

        #Fill up the verification_images folder with images to test the input_image against
        if cv2.waitKey(1) & 0xFF == ord("v"):
            cv2.imwrite(os.path.join("application_data", "verification_images", "{}.jpg".format(uuid.uuid1())), face_resized)

        #Save the image to input_image
        if cv2.waitKey(1) & 0xFF == ord("t"):
            cv2.imwrite(os.path.join("application_data", "input_image", "input_image.jpg"), face_resized)
            results, verified = verification(model, 0.855, 0.6)
            if verified == True:
                print("You are verified")
            else:
                print("Unverified")

    #Exit when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()